In [71]:
import pandas as pd


stations = pd.read_csv('post_locations - Sheet1.csv')
points = pd.read_csv('Points - Sheet1.csv')
points.head()

,Number,Adrress,Lat,Long
0,1,"София, ул. Монтевидео 51",42.687105,23.255028
1,2,"София, Околовръстен път 214",42.624744,23.352466
2,3,"София, бул. Бъкстон бл. 208",42.670188,23.277321
3,4,"София, ул.Св.Св.Кирил и Методий 124",42.703933,23.317588
4,5,"София, ул.5001",42.662359,23.417433


In [59]:
import geopandas as gpd

def create_gdf(df, x='Lat', y='Long'):
    return gpd.GeoDataFrame(df, 
    geometry=gpd.points_from_xy(df[y], df[x]), 
             crs={'init':'EPSG:4326'})

stations_gdf = create_gdf(stations)
points_gdf = create_gdf(points)

stations_gdf.head()

,PostCode,Name,City,Address,Lat,Long,geometry
0,1000,ПС София Ц - колетна сл.,София,"София, Ул. ""Ген. Гурко"" №6",42.693437,23.325002,POINT (23.32500 42.69344)
1,1101,ПС София 13 ИРМ КАТ,София,"София, Бул. ""Драган Цанков"" №31А",42.666055,23.354919,POINT (23.35492 42.66606)
2,1111,ПС София 11,София,"София, Ул. Гео Милев 18",42.684823,23.357099,POINT (23.35710 42.68482)
3,1113,ПС София 13,София,"София, Бул.Цариградско Шосе №85.Бл.109",42.676210,23.359021,POINT (23.35902 42.67621)
4,1124,ПС София 124,София,"София, Ул. Хан Омуртаг 84",42.686692,23.351005,POINT (23.35100 42.68669)


In [62]:
import folium 

m = folium.Map([42.697708, 23.321867], tiles='CartoDb dark_matter')
locs_stations = zip(stations_gdf.Lat, stations_gdf.Long)
locs_points = zip(points_gdf.Lat, points_gdf.Long)
for location in locs_stations:
    folium.CircleMarker(location=location, 
        color='red',   radius=4).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
        color='white', radius=2).add_to(m)
m

In [70]:
import shapely
from shapely.ops import nearest_points
def calculate_nearest(row, destination, val, col='geometry'):
    # 1 - create unary union    
    dest_unary = destination['geometry'].unary_union
    # 2 - find closest point
    nearest_geom = nearest_points(row[col], dest_unary)
    # 3 - Find the corresponding geom
    match_geom = destination.loc[destination.geometry 
                == nearest_geom[1]]
    # 4 - get the corresponding value
    match_value = match_geom[val].to_numpy()[0]
    return match_value

# Get the nearest geometry
points_gdf['nearest_geom'] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val='geometry', axis=1)
# Get the nearest Bike station name
points_gdf['nearest_station'] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val='Name', axis=1)
points_gdf.head()

,Number,Adrress,Lat,Long,geometry,nearest_geom,nearest_station
0,1,"София, ул. Монтевидео 51",42.687105,23.255028,POINT (23.25503 42.68710),POINT (23.2528865 42.67733270000001),ПС София 18 ИРМ НБУ
1,2,"София, Околовръстен път 214",42.624744,23.352466,POINT (23.35247 42.62474),POINT (23.368862 42.61981979999999),ПС София 66 ИРМ
2,3,"София, бул. Бъкстон бл. 208",42.670188,23.277321,POINT (23.27732 42.67019),POINT (23.2725628 42.6713754),ПС София 18
3,4,"София, ул.Св.Св.Кирил и Методий 124",42.703933,23.317588,POINT (23.31759 42.70393),POINT (23.3219514 42.6991417),ПС София 77
4,5,"София, ул.5001",42.662359,23.417433,POINT (23.41743 42.66236),POINT (23.4083304 42.6485511),ПС София 82


In [79]:
from shapely.geometry import LineString
# Create LineString Geometry
points_gdf['line'] = points_gdf.apply(lambda row: LineString([row['geometry'], row['nearest_geom']]), axis=1)
# Create Line Geodataframe
line_gdf = points_gdf[['Number', 'nearest_station', 'line']].set_geometry('line')
# Set the Coordinate reference
line_gdf.crs = crs={'init':'epsg:4326'}

m = folium.Map([42.697708, 23.321867],zoom_start = 12,  
    tiles='CartoDb dark_matter')
locs_stations = zip(stations_gdf.Lat, stations_gdf.Long)
locs_points = zip(points_gdf.Lat, points_gdf.Long)
for location in locs_stations:
    folium.CircleMarker(location=location, 
              color='red',  radius=8).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
              color='white', radius=4).add_to(m)
folium.GeoJson(line_gdf).add_to(m)
m.save('map2.html')
m

C:\Users\Dimitar Kumenov\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
